In [3]:
import pandas as pd
import sklearn as sk

# Read Data & create a list of your columns 

In [4]:
df=pd.read_csv('train.csv')
df.head()

,Questions,Answers
0,What is Data Analytics ?,Data analytics is the process of analyzing raw...
1,What does this project do ?,This is a chat-bot based on cosine simarilari...


In [5]:
print(df.columns)
df.columns = df.columns.str.strip()

Index(['Questions ', 'Answers'], dtype='object')


In [6]:
answers_list=df['Answers'].tolist()

In [7]:
questions_list=df['Questions'].tolist()

In [8]:
print(answers_list)

['Data analytics is the process of analyzing raw data to find trends and answer questions.', 'This is a chat-bot based on cosine  simarilarity with the integration of data anlytics which answers your question.']


# Importing libraries

In [9]:
!pip install nltk

import nltk 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\RUDRA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\RUDRA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\RUDRA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\RUDRA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import re

def preprocess(text):
    lemmatizer=WordNetLemmatizer()
    stemmer=PorterStemmer()
    text=re.sub(r'[^\w\s]','',text) # Remove non-alphanumeric charaters 

    tokens=nltk.word_tokenize(text.lower())
    tokens=[token for token in tokens if token not in stopwords.words('english')]
    lemmatized_tokens=[lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens=[stemmer.stem(token) for token in lemmatized_tokens]
    return " ".join(stemmed_tokens)

In [11]:
def preprocess_with_stopwords(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

In [12]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize,token_pattern=None)
X = vectorizer.fit_transform([preprocess(q) for q in questions_list])

In [14]:
vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize,token_pattern=None)
X = vectorizer.fit_transform([preprocess(q) for q in questions_list])

def get_response(text):
    processed_text = preprocess_with_stopwords(text)
    print("processed_text:", processed_text)
    vectorized_text = vectorizer.transform([processed_text])
    similarities = cosine_similarity(vectorized_text, X)
    print("similarities:", similarities)
    max_similarity = np.max(similarities)
    print("max_similarity:", max_similarity)
    if max_similarity > 0.6:
        high_similarity_questions = [q for q, s in zip(questions_list, similarities[0]) if s > 0.6]
        print("high_similarity_questions:", high_similarity_questions)

        target_answers=[]
        for q in high_similarity_questions:
            q_index=questions_list.index(q)
            target_answers.append(answers_list[q_index])
        print(target_answers)

        Z = vectorizer.fit_transform([preprocess_with_stopwords(q) for q in high_similarity_questions])
        processed_text_with_stopwords = preprocess_with_stopwords(text)
        print("processed_text_with_stopwords:", processed_text_with_stopwords)
        vectorized_text_with_stopwords = vectorizer.transform([processed_text_with_stopwords])
        final_similarities = cosine_similarity(vectorized_text_with_stopwords, Z)
        closest = np.argmax(final_similarities)
        return target_answers[closest]
    else:
        return "I can't answer this question."

get_response('What is data analytics?')

processed_text: data analytics
similarities: [[0.70710678 0.        ]]
max_similarity: 0.7071067811865476
high_similarity_questions: ['What is Data Analytics ?']
['Data analytics is the process of analyzing raw data to find trends and answer questions.']
processed_text_with_stopwords: data analytics


'Data analytics is the process of analyzing raw data to find trends and answer questions.'